In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

USED IN THIS NOTEBOOK:

* REMOVED OUTLIERS (try with z-score next time?)
* SMOTE to remove dataset imbalance
* 
* XGBOOST
* RANDOMFOREST
* 
* RANDOMIZEDGRIDSEARCH

In [ ]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgbm
import catboost as cb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score, accuracy_score


In [ ]:
df = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.head()

In [ ]:
cols = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']

In [ ]:
!pip install sweetviz

import sweetviz as sv
data_report = sv.analyze(df)
data_report.show_html('Analysis.html')

from IPython.display import IFrame
IFrame(src = 'Analysis.html',width=1000,height=600)

In [ ]:
df.describe()

# FEATURE ENGINEERING

In [ ]:
num_cols = df[['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'time']]
cat_cols = df[['anaemia', 'diabetes', 'high_blood_pressure', 'serum_sodium', 'sex', 'smoking']]

# **removing outliers**

In [ ]:
a = (df["serum_creatinine"] < 5).value_counts()
print(a)

In [ ]:
df = df.drop(df[df['creatinine_phosphokinase'] > 3100].index)
df = df.drop(df[df["ejection_fraction"] > 65].index)
df = df.drop(df[df["platelets"] > 650000].index)
df = df.drop(df[df["serum_sodium"] < 120].index)
df = df.drop(df[df["serum_creatinine"] > 5].index)

print(df.shape)

In [ ]:
y = df['DEATH_EVENT']
X = df.drop('DEATH_EVENT', axis=1)

In [ ]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, random_state=0)

# **normalizing/scaling numeric data**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)

#return to pandas dataframe format and add column names
X_train = pd.DataFrame(X_train, columns=cols)
X_valid = pd.DataFrame(X_valid, columns=cols)

X_train.head()

In [ ]:
for i in X_train.columns:
    plt.hist(X_train[i])
    plt.title(i)
    plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE

print(X_train.shape)
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)
print(X_train.shape)


# Model Building

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
# setting a Baseline
gnb = GaussianNB()
cv = cross_val_score(gnb,X_train,y_train,cv=10)
print(cv)
print(cv.mean())

Of the models used, XGB and RandomForest gave the best results, we will continue with these
(deleted the other model codes for clarity)

# Version 3: feature selection before training models

Forward selection is a wrapper model that evaluates the predictive power of the features jointly and returns a set of features that performs the best. It selects the predictors one by one and chooses that combination of features that makes the model perform the best based on the cumulative residual sum of squares. This process continues till there is no impact on the performance of the model with the incoming variable.

In [ ]:
print(X_train.shape)


from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector
forward_feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1),
           k_features=6,
           forward=True,
           verbose=2,
           scoring='roc_auc',
           cv=5)
rfselector = forward_feature_selector.fit(X_train,y_train, custom_feature_names=cols)
print(rfselector.k_feature_names_)

X_train_rf = rfselector.transform(X_train)

print(X_train_rf.shape)


In [ ]:
rf_cols=rfselector.k_feature_names_
#transform to a list for later use
rf_cols = list(rf_cols)
print(rf_cols)


In [ ]:
X_train_rf = pd.DataFrame(X_train_rf)
X_train_rf.columns = rfselector.k_feature_names_
X_valid_rf = X_valid[rf_cols]

In [ ]:
print(X_train.shape)


from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector
forward_feature_selector = SequentialFeatureSelector(XGBClassifier(random_state =1,use_label_encoder=False),
           k_features=6,
           forward=True,
           verbose=2,
           scoring='roc_auc',
           cv=5)
xgbselector = forward_feature_selector.fit(X_train,y_train, custom_feature_names=cols)
print(xgbselector.k_feature_names_)

X_train_xgb = xgbselector.transform(X_train)

print(X_train_xgb.shape)

In [ ]:
xgb_cols = xgbselector.k_feature_names_

#transform to a list for later use
xgb_cols = list(xgb_cols)

In [ ]:
X_train_xgb = pd.DataFrame(X_train_xgb)
X_train_xgb.columns = xgbselector.k_feature_names_

X_valid_xgb = X_valid[xgb_cols]

X_valid_xgb.head()


In [ ]:
#rf = RandomForestClassifier(random_state = 1)
#cv = cross_val_score(rf,X_train_rf,y_train,cv=10)
#print(cv)
#print(cv.mean())

In [ ]:
#from xgboost import XGBClassifier
#xgb = XGBClassifier(random_state =1,use_label_encoder=False)
#cv = cross_val_score(xgb,X_train,y_train,cv=10)
#print(cv)
#print(cv.mean())

# Fine-Tuning parameters

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV 

In [ ]:
#simple performance reporting function
def clf_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

In [ ]:
rf = RandomForestClassifier(random_state = 1)

param_grid = {"criterion" : ["gini", "entropy"],
              "n_estimators": [i for i in range(1, 250, 50)],
              "max_depth": [i for i in range(5, 25, 2)],
              "min_samples_split" : [3, 12, 20, 25, 35],
              'oob_score' : [True],
              'random_state' : [1],
              'n_jobs' : [-1]}

rf_clf = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, cv = 10, scoring = 'f1', n_jobs=-1)
rf_clf.fit(X_train_rf, y_train)

clf_performance(rf_clf,'Random Forest')

#RESULT NO SMOTE
#Best Score: 0.7447237309306274
#Best Parameters: {'criterion': 'gini', 'max_depth': 11, 'min_samples_split': 12, 'n_estimators': 101, 'n_jobs': -1, 'oob_score': True, 'random_state': 1}
#RESULT SMOTE
#Best Score: 0.9072604952727904
#Best Parameters: {'random_state': 1, 'oob_score': True, 'n_jobs': -1, 'n_estimators': 51, 'min_samples_split': 3, 'max_depth': 15, 'criterion': 'gini'}

In [ ]:
xgb = XGBClassifier(random_state =1,use_label_encoder=False)

parameter_grid = {
    'num_boost_round': [10, 25, 50],
    'eta': [0.05, 0.1, 0.3],
    'max_depth': [3, 4, 5],
    'subsample': [0.9, 1.0], 
    'colsample_bytree': [0.9, 1.0],
    }
    
xgb_clf = RandomizedSearchCV(estimator=xgb, param_distributions=parameter_grid, cv = 5, scoring = 'f1', n_jobs=-1)
xgb_clf.fit(X_train_xgb, y_train)

clf_performance(xgb_clf,'XGB')

#RESULT NO SMOTE
#Best Score: 0.7505170604081675
#Best Parameters: {'colsample_bytree': 0.9, 'eta': 0.3, 'max_depth': 3, 'num_boost_round': 10, 'subsample': 1.0}
#RESULT SMOTE (randomized search)
#Best Score: 0.8880470174060274
#Best Parameters: {'subsample': 0.9, 'num_boost_round': 50, 'max_depth': 5, 'eta': 0.05, 'colsample_bytree': 1.0}


# Use best parameter combinations for final prediction

In [ ]:
rf_clf.best_params_

In [ ]:
clf = RandomForestClassifier(random_state= 1,
 oob_score= True,
 n_jobs= -1,
 n_estimators= 101,
 min_samples_split= 3,
 max_depth= 13,
 criterion= 'entropy')

clf.fit(X_train_rf, y_train)
y_pred = clf.predict(X_valid_rf)

from sklearn.metrics import f1_score
print(f1_score(y_valid, y_pred))
print(accuracy_score(y_valid, y_pred)) 


In [ ]:
xgb_clf.best_params_

In [ ]:
clf = XGBClassifier(subsample= 1.0,
 num_boost_round= 50,
 max_depth= 3,
 eta= 0.3,
 colsample_bytree= 0.9)
clf.fit(X_train_xgb, y_train)
y_pred = clf.predict(X_valid_xgb)

print(f1_score(y_valid, y_pred))
print(accuracy_score(y_valid, y_pred))